## Original model

This notebook contains an implementation of the original model from Swaddle (2017) adapted to use the full complex form of unitaries. It has been adapted in order that both the GRU global decomposition network and fully-connected local decomposition network form a sequence of networks that are called sequentially.

Code has also been added to calculate the fidelity of estimated unitary sequences against ground-truth unitary sequences.

In [1]:
import numpy as np
import scipy as sc
import qutip as qt
from matplotlib import pyplot as plt
import random as rn
%matplotlib notebook

In [1]:

# Preamble
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,optimizers,constraints,initializers,Model,backend
import pickle
import time
import zipfile    
import os


import qutip as qt
import pandas as pd
import scipy as sc
import random as rn
import os
import itertools as itr
from itertools import combinations
from operator import itemgetter
import operator
from functools import reduce
import csv

from tensorflow.python.ops import control_flow_ops

from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM, LSTMCell, GRU

from tensorflow.keras.callbacks import CSVLogger
from numpy import array
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model


from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from matplotlib import animation
from IPython.display import HTML, Image
%matplotlib inline
plt.style.use('seaborn-white')
import pickle
from flattenunitary import flattenunitary
from flattenunitary import realise
from dataprocess import dataprocess


In [4]:
from customlayers import *

In [5]:
'''Load dataset'''
import pickle
su2dim = 2
nseg = 10
ntrain = 1000
hscale = 0.1 #\delta tj (i.e. h in code or duration of pulse)
su2ngen = None #0: \Lambda_0 in \Delta; 1: \Lambda_0 \in su2n
holo = None #whether training data to be generated from Boozer

with open('su{0}_ntrain{1}_nseg{2}_hscale{3}_su2gen{4}.pickle'
          .format(su2dim,ntrain, nseg, hscale, su2ngen), 'rb') as handle:
    gentest = pickle.load(handle)

'''Convert dictionary to tuple so behaves like an object once loaded'''
from collections import namedtuple
gentest = namedtuple('Struct', gentest.keys())(*gentest.values())

In [6]:
'''=========Real and imaginary parts of (Uj)======'''

Ujseq_real = (tf.math.real(np.asarray(gentest.Uj_master_array)))
Ujseq_imag = (tf.math.imag(np.asarray(gentest.Uj_master_array)))
Ujseq = tf.convert_to_tensor(np.asarray(gentest.Uj_master_array))
Ujinputseq = np.asarray(gentest.Uj_master_realised)

In [7]:
'''Recreate xtrain function from QMLcontrol model: constructs U_T by accumulating (U_j).'''
from itertools import accumulate
Ujarray = np.asarray(gentest.Uj_master_realised)
def xtrainmethod(Ujarray):
    evolve = lambda U_j,U: U @ U_j
    acclist = []
    for list1 in Ujarray:
        acclist.append(list(accumulate(list1, evolve))[-1])
    xtrain = tf.convert_to_tensor(np.asarray(acclist))
    return xtrain
xtrain = xtrainmethod(Ujarray)

In [13]:
'''==============GRU: | Global decomposition model | Input: UT | Output: (Uj) =================='''
'''GRU: combining basic GRU model from Swaddle'''
'''Inputs: the target unitary (U_T), output is global decomposition (U_j)'''

Ujinputseq = np.asarray(gentest.Uj_master_realised)
Ujinputseq.shape
Ujcoefinput = tf.convert_to_tensor(np.asarray(gentest.Uj_master_coef_list), dtype=tf.float64)

_su2n_realdim = gentest.su2n_dim * 2
seg = gentest.segments
ntrain = gentest.ntraining
deltadim = np.asarray(gentest.su_2n_delta).shape[0]

coefdim = seg * deltadim
'''In QMLmodel class coefdim is: self.coef_numb_seg'''
Ujcoeflabel = np.asarray(gentest.Uj_master_coef_list)
tot = deltadim * gentest.segments

origdim = (xtrain.shape[1], xtrain.shape[2])
tf.print('origdim')
tf.print(origdim)
fidones_seg = tf.ones((gentest.ntraining,gentest.segments))
tf.print('reshape shape')
tf.print(seg * _su2n_realdim * _su2n_realdim)
original_inputs = tf.keras.Input(shape=origdim, name='UTnput', batch_size=None)
y1 = layers.Reshape((-1,_su2n_realdim *_su2n_realdim,))(original_inputs)

totalflat = gentest.segments*gentest.su2n_dim*gentest.su2n_dim
GRUreal = layers.GRU(totalflat, return_sequences=True, activation='tanh', name='GRUreal')(y1)
GRUimag = layers.GRU(totalflat, return_sequences=True, activation='tanh', name='GRUimag')(y1)
Reshapereal = layers.Reshape((gentest.segments,gentest.su2n_dim,gentest.su2n_dim,),name='Reshapereal')(GRUreal)
Reshapeimag = layers.Reshape((gentest.segments,gentest.su2n_dim,gentest.su2n_dim,), name='Reshapeimag')(GRUimag)

'''GRU model'''
'''MSE flattened input Ujseq for loss'''
grumod2 = Model(inputs=original_inputs, outputs=[Reshapereal,Reshapeimag], name='seqmod')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
MSE = tf.keras.losses.MeanSquaredError()
grumod2.compile(optimizer, loss=[MSE,MSE])
   
'''=======Train global GRU model============='''
epochs=100
csv_logger = CSVLogger('origgru1.csv')
h = grumod2.fit(xtrain,[Ujseq_real,Ujseq_imag],epochs=epochs, verbose=1, 
                validation_data=(xtrain,[Ujseq_real,Ujseq_imag]),
                batch_size=10, validation_split = 0.25, shuffle=True,callbacks=[csv_logger])


'''=======Prediction: use model to predict unitaries (global decomposition)=========='''
grutens = grumod2.predict([xtrain, Ujseq_real,Ujseq_imag])
tfconvert = tf.convert_to_tensor(grutens)
freal = tfconvert[0]
fimag = tfconvert[1]


'''==============FC model: | Local decomposition model | Input: (U_j) | Output: (c_j) =================='''
'''Description: trained on coefficients. Reuses above layers.'''
inputUre = layers.Input(shape=(gentest.segments,gentest.su2n_dim,gentest.su2n_dim,), name='input2', batch_size=None)
inputUim = layers.Input(shape=(gentest.segments,gentest.su2n_dim,gentest.su2n_dim,), name='input3', batch_size=None)
concat = layers.Concatenate(axis=-1,name='concatlayer')([inputUre,inputUim])
flatten = layers.Flatten()(concat)
tot = deltadim * gentest.segments
d1 = layers.Dense(tot, activation='tanh', name='Dense1')(flatten)
d1 = layers.Dropout(0.2)(d1)
d1 = layers.Dense(tot, activation='tanh', name='Dense2')(d1)
d1 = layers.Dropout(0.2)(d1)
d1 = layers.Dense(tot, activation='tanh', name='Dense3')(d1)
d1 = layers.Dropout(0.2)(d1)
d1 = layers.Dense(tot, activation='tanh', name='Dense4')(d1)
coefmod = Model(inputs=[inputUre,inputUim], outputs=d1, name='coefmod')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
coefmod.compile(optimizer, loss=MSE)
#epochs=2
csv_logger = CSVLogger('orig_su8d_FC.csv')
Ujcoefinput = tf.convert_to_tensor(np.asarray(gentest.Uj_master_coef_list), dtype=tf.float64)
shape = (-1,Ujcoefinput.shape[1] * Ujcoefinput.shape[2])
Ujcoeflabel = tf.reshape(Ujcoefinput,shape)
'''=======train: train model (local decomposition)=========='''
h1 = coefmod.fit([Ujseq_real,Ujseq_imag],Ujcoeflabel,epochs=epochs, verbose=1, 
                 batch_size=10, validation_split = 0.25, shuffle=True,callbacks=[csv_logger])


'''=======Prediction: use model to predict coefficients (controls)=========='''
coefpred = coefmod.predict([freal,fimag])



'''===========Model: Reconstruct unitaries from estimated unitaries using
generators, then compare fidelity with original sequence (Uj)'''
seg = gentest.segments
from customlayers import ham_est_simple,unitary_simple
ham_est_layer_simple = ham_est_simple(gentest.su_2n_delta,seg, name='ham_layer')
su2nidentity = tf.convert_to_tensor(np.identity(gentest.su2n_dim), dtype=np.complex128)
unitary_simp = unitary_simple(seg, su2nidentity, gentest.su2n_dim, name='unitaryreconstruct')
coefinput = layers.Input(shape=(coefpred.shape[1],), name='coefinput', batch_size=None)
h1 = ham_est_layer_simple(coefinput)
u1 = unitary_simp(h1)
unitarymod = Model(inputs=coefinput, outputs=u1, name='coefmod')  
#unitarymod.summary()
unitarymodpred = unitarymod.predict(coefpred)
'''Note: unitarymod is not trained.'''

'''Now check using fidelity model above'''

unconvert = tf.convert_to_tensor(unitarymodpred)
unconvert.shape
unreal = tf.math.real(unconvert)
unimag = tf.math.imag(unconvert)

'''=======Second model, check fidelity of this output=========='''
tfconvert = tf.convert_to_tensor(grutens)
shp = tfconvert.shape
origshape = (shp[2],shp[3],shp[4])
fidreal = layers.Input(shape=origshape, name='fidinput1', batch_size=None)
fidimag = layers.Input(shape=origshape, name='fidinput2', batch_size=None)
y = recombcomplex(name='unitaryoutput')([fidreal,fidimag])

inputUre = layers.Input(shape=(gentest.segments,gentest.su2n_dim,gentest.su2n_dim,), name='input2', batch_size=None)
inputUim = layers.Input(shape=(gentest.segments,gentest.su2n_dim,gentest.su2n_dim,), name='input3', batch_size=None)
y_fid = fidmulti_unitary(gentest.su2n_dim, name='finalfidelity')([inputUre,inputUim,y])
 
fidmod = Model(inputs=[fidreal,fidimag,inputUre,inputUim], outputs=y_fid, name='fidmod')

'''=========Calculate fidelities for estimated (\hat{U}j) that are constructed using the
coefficients above and generators with the ground-trut (U_j). Store as a tensor of fidelities.'''
uncheck = fidmod([unreal,unimag,Ujseq_real,Ujseq_imag])
print(uncheck)
'''Find average operator fidelity across examples'''
avf = tf.reduce_mean(uncheck).numpy()
print(avf)

origdim
(4, 4)
reshape shape
160
Epoch 1/100
75/75 [==============================] - 1s 8ms/step - loss: 0.4446 - Reshapereal_loss: 0.4393 - Reshapeimag_loss: 0.0054 - val_loss: 0.3511 - val_Reshapereal_loss: 0.3502 - val_Reshapeimag_loss: 9.0110e-04
Epoch 2/100
75/75 [==============================] - 0s 2ms/step - loss: 0.2774 - Reshapereal_loss: 0.2771 - Reshapeimag_loss: 3.3751e-04 - val_loss: 0.2090 - val_Reshapereal_loss: 0.2089 - val_Reshapeimag_loss: 8.4841e-05
Epoch 3/100
75/75 [==============================] - 0s 2ms/step - loss: 0.1598 - Reshapereal_loss: 0.1598 - Reshapeimag_loss: 4.2224e-05 - val_loss: 0.1180 - val_Reshapereal_loss: 0.1180 - val_Reshapeimag_loss: 2.3001e-05
Epoch 4/100
75/75 [==============================] - 0s 2ms/step - loss: 0.0917 - Reshapereal_loss: 0.0917 - Reshapeimag_loss: 1.9774e-05 - val_loss: 0.0701 - val_Reshapereal_loss: 0.0701 - val_Reshapeimag_loss: 1.8134e-05
Epoch 5/100
75/75 [==============================] - 0s 2ms/step - loss: 0.0566

75/75 [==============================] - 0s 2ms/step - loss: 9.0451e-04 - Reshapereal_loss: 9.0264e-04 - Reshapeimag_loss: 1.8753e-06 - val_loss: 8.7878e-04 - val_Reshapereal_loss: 8.7706e-04 - val_Reshapeimag_loss: 1.7184e-06
Epoch 38/100
75/75 [==============================] - 0s 2ms/step - loss: 8.5125e-04 - Reshapereal_loss: 8.4950e-04 - Reshapeimag_loss: 1.7445e-06 - val_loss: 8.2767e-04 - val_Reshapereal_loss: 8.2595e-04 - val_Reshapeimag_loss: 1.7138e-06
Epoch 39/100
75/75 [==============================] - 0s 2ms/step - loss: 8.0179e-04 - Reshapereal_loss: 8.0015e-04 - Reshapeimag_loss: 1.6357e-06 - val_loss: 7.8005e-04 - val_Reshapereal_loss: 7.7853e-04 - val_Reshapeimag_loss: 1.5188e-06
Epoch 40/100
75/75 [==============================] - 0s 2ms/step - loss: 7.5603e-04 - Reshapereal_loss: 7.5450e-04 - Reshapeimag_loss: 1.5280e-06 - val_loss: 7.3555e-04 - val_Reshapereal_loss: 7.3417e-04 - val_Reshapeimag_loss: 1.3879e-06
Epoch 41/100
75/75 [==============================] -

75/75 [==============================] - 0s 2ms/step - loss: 1.5980e-04 - Reshapereal_loss: 1.5960e-04 - Reshapeimag_loss: 1.9971e-07 - val_loss: 1.5652e-04 - val_Reshapereal_loss: 1.5633e-04 - val_Reshapeimag_loss: 1.8670e-07
Epoch 72/100
75/75 [==============================] - 0s 2ms/step - loss: 1.5269e-04 - Reshapereal_loss: 1.5249e-04 - Reshapeimag_loss: 1.9237e-07 - val_loss: 1.4963e-04 - val_Reshapereal_loss: 1.4947e-04 - val_Reshapeimag_loss: 1.5971e-07
Epoch 73/100
75/75 [==============================] - 0s 2ms/step - loss: 1.4594e-04 - Reshapereal_loss: 1.4577e-04 - Reshapeimag_loss: 1.7031e-07 - val_loss: 1.4298e-04 - val_Reshapereal_loss: 1.4281e-04 - val_Reshapeimag_loss: 1.6935e-07
Epoch 74/100
75/75 [==============================] - 0s 2ms/step - loss: 1.3950e-04 - Reshapereal_loss: 1.3933e-04 - Reshapeimag_loss: 1.7150e-07 - val_loss: 1.3672e-04 - val_Reshapereal_loss: 1.3653e-04 - val_Reshapeimag_loss: 1.8663e-07
Epoch 75/100
75/75 [==============================] -

75/75 [==============================] - 0s 1ms/step - loss: 0.0280 - val_loss: 0.0062
Epoch 12/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0287 - val_loss: 0.0068
Epoch 13/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0267 - val_loss: 0.0054
Epoch 14/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0256 - val_loss: 0.0065
Epoch 15/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0266 - val_loss: 0.0078
Epoch 16/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0255 - val_loss: 0.0071
Epoch 17/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0253 - val_loss: 0.0065
Epoch 18/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0265 - val_loss: 0.0054
Epoch 19/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0242 - val_loss: 0.0116
Epoch 20/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0239 - val_loss: 0.0080
Epoch 21/100


75/75 [==============================] - 0s 1ms/step - loss: 0.0168 - val_loss: 0.0064
Epoch 93/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0184 - val_loss: 0.0063
Epoch 94/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0183 - val_loss: 0.0069
Epoch 95/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0177 - val_loss: 0.0053
Epoch 96/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0202 - val_loss: 0.0076
Epoch 97/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0184 - val_loss: 0.0057
Epoch 98/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0185 - val_loss: 0.0064
Epoch 99/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0166 - val_loss: 0.0068
Epoch 100/100
75/75 [==============================] - 0s 1ms/step - loss: 0.0189 - val_loss: 0.0062
tf.Tensor(
[[0.87568626 0.87569706 0.87570128 ... 0.87570081 0.87569038 0.87568804]
 [0.8999335  0.89992944 0.89